Plan of attack: 

-Subset THE DEMOGRAPHICS dataset by: 
Date Enrolled in START (Before and after March 1, 2020)
Gender (Male/Female)
Race 

-Merge the demographics data and the Start FEIS data by patient ID # 
-Clean data so only relevant columns are left (Demographic data + family input)

-First conduct sentiment analysis on each subset to see which feedback is the most negative (could be an indication of which subset is getting the least amount of care)

-Then conduct topic modeling on the subsets to see where families think care is lacking 


In [2]:
# Importing modules
import pandas as pd

In [24]:
demo_df = pd.read_excel(r"../files/Dartmouth_Data_Set.xlsx")
FEIS_df = pd.read_excel(r"../files/START_FEIS_Data.xlsx")
time_df = pd.read_excel(r"../files/Dartmouth_Time_Data.xlsx")
dict_df = pd.read_excel(r"../files/Final SIRS_Data_Dictionary_V13.1 October 2020.xlsx")

In [51]:
# Cleaning the demographics dataset
demographics = demo_df[['Local ID', 'Region', 'Date Enrolled in START', 'Gender', 'Race', 'Date of birth', 'Ethnicity',
                              'Level of Intellectual Disability', 'Psychiatric diagnoses', 'Medical diagnoses', 'Other Disabilities',
                              'Funding']]

In [46]:
# Merging datasets
merged = pd.merge(demographics, FEIS_df, how = 'inner', left_on = ['Local ID'], right_on = ['Respondent ID #  (SIRS Local ID)'])

# Look at type of join (changed)
merged

,Local ID,Region,Date Enrolled in START,Gender,Race,Date of birth,Ethnicity,Level of Intellectual Disability,Psychiatric diagnoses,Medical diagnoses,...,"In\nthe past year, did your family member use in-patient psychiatric services?","If\nyes, were the inpatient services that your family member received helpful to\nhim/her in your opinion? ?",How\nmuch help was available to you at night or on weekends if your family member\nhad a crisis?,Are\nthere options outside of the hospital for individuals experiencing a crisis to\ngo for help (i.e. crisis/hospital diversion beds)?,Who\nwas the primary source of information about your family memberâ€™s mental health\nservices?,"If other, please describe..2","During the past year, how much involvement\ndid you want to have in your family memberâ€™s treatment plan?",Was there any particular service that your\nfamily member needed that was not available?,"If yes, please describe the service.",What\nadvice would you give to service planners regarding the mental health service\nneeds of persons with IDD and their families?
0,8008815,California : CA START East Bay,2020-12-29,Male,Other: Mexican,2006-11-13,Hispanic - specific origin not specified,Mild,"Attention-Deficit/Hyperactivity Disorder, Auti...",NaN,...,No,NaN,None at all,None at all,His/her therapist,NaN,A lot,No,NaN,NaN
1,6570649,California : CA START San Andreas,2020-12-29,Female,"Unknown, not collected",1999-02-07,Hispanic - specific origin not specified,Severe,"Generalized Anxiety Disorder, Schizophrenia","Hematology/Oncology, Other: Digeorge syndrome",...,Yes,All that was wanted/needed,Did not know/answer,"Some, but not as much as was needed/wanted",Other,Santa Cruz County Psychiatrist,A lot,Yes,A counselor was not and has not been made avai...,â€œPlease be aware of her conditions and diagn...
2,434021,New York : Region 3,2020-12-28,Female,White,2001-12-24,Not of Hispanic origin,Borderline,"Attention-Deficit/Hyperactivity Disorder, Auti...",Gastro/Intestinal,...,No,NaN,None at all,None at all,His/her psychiatrist,NaN,A lot,Yes,In-home behavior support,NaN
3,6580618,California : CA START San Andreas,2020-12-23,Male,White,2004-12-21,Not of Hispanic origin,Moderate,NaN,NaN,...,No,NaN,Very little,None at all,Other,school based service providers,A lot,Yes,"""After Trevorâ€™s psychiatrist left the office...",Declined to answer/did not know.
4,354280,New York : Region 3,2020-12-21,Male,"Unknown, not collected",2005-09-04,"Unknown, not collected",Mild,"Attention-Deficit/Hyperactivity Disorder, Auti...","Gastro/Intestinal, Immunology/Allergy",...,No,NaN,None at all,None at all,His/her service facilitator,NaN,A lot,Yes,At home off hour support on phone or in person...,"Listen to the parents, take what parents repor..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,1013197,Texas : Tarrant County,2014-06-05,Male,Black or African American,1997-03-01,Not of Hispanic origin,Moderate,"Autism Spectrum Disorder (2014), Schizoaffecti...","Pulmonary disorders, Sleep Disorder/Concern",...,Yes,NaN,"Some, but not as much as was needed/wanted",All that was wanted/needed,Your family member him/herself,NaN,A lot,No,NaN,NaN
1093,1100502,Texas : Tarrant County,2014-05-09,Female,White,2004-06-01,Not of Hispanic origin,Mild,"Attention-Deficit/Hyperactivity Disorder, Bipo...",NaN,...,No,None at all,All that was wanted/needed,All that was wanted/needed,His/her service facilitator,NaN,Some,No,NaN,NaN
1094,1132230,Texas : Tarrant County,2014-05-02,Female,Black or African American,1995-04-01,Not of Hispanic origin,Moderate,"Attention-Deficit/Hyperactivity Disorder, Bipo...",NaN,...,Yes,"Some, but not as much as was needed/wanted",Very little,"Some, but not as much as was needed/wanted",Your family member him/herself,NaN,A lot,Yes,Wraparound services and continuily of care,NaN
1095,11128011,New Hampshire : Region 5 - Monadnock Developme...,2012-02-16,Male,White,1989-06-01,Not of Hispanic origin,Profound,"Anxiety Disorder

In [64]:
# Subsetting by gender
demographics_male = merged.loc[merged['Gender']=='Male']
demographics_female = merged.loc[merged['Gender']=='Female']

# Subsetting by race
male_white = demographics_male[demographics_male['Race'] == "White"]
male_nonwhite = demographics_male[demographics_male['Race'] != "White"]

female_white = demographics_female[demographics_female['Race'] == "White"]
female_nonwhite = demographics_female[demographics_female['Race'] != "White"]


#male_white.shape
#male_nonwhite.shape

#female_white.shape
#female_nonwhite.shape

(114, 69)